# From Bloqade Squin kernels to Stim

In [ ]:
import io
from kirin import ir
from bloqade.stim.emit import EmitStimMain
from bloqade.stim.passes import SquinToStimPass

# circuit emitter
def codegen(mt: ir.Method):
    # method should not have any arguments!
    buf = io.StringIO()
    emit = EmitStimMain(dialects=bloqade_stim.main, io=buf)
    emit.initialize()
    emit.run(mt)
    return buf.getvalue().strip()

# language compiler and converter
def to_stim(mt: ir.Method):
    SquinToStimPass(mt.dialects)(mt)
    return codegen(mt)

# Squin gates of interest

given immutable lists `q: IList[Qubit, Literal[7]]` of qubits, and `p: float` noise rate

### Depolarizing noise channels

In [ ]:
from bloqade import squin as sq

# Single qubit noise; broadcast applies channel in parallel to all qubits in list 
sq.broadcast.depolarize(p, [q[0], q[1]])

# Two-qubit noise; broadcast applies channel in parallel to all qubits in lists 
sq.broadcast.depolarize2(p, q1, q2)

### Measure and reset

In [ ]:
# measure
sq.broadcast.measure(q)

# reset
sq.broadcast.reset(q)

note that to implement resets and measurements on single qubits, just drop `.broadcast` and apply to single-qubit registers.

### Convert circuit to Stim/Tsim, visualize, run sampler

In [ ]:
import stim
import tsim

# conversion pipeline
@squin.kernel
def main():
    my_kernel()

stim_code = to_stim(main)
stim_circ = tsim.Circuit(stim_code)

# plotting circuit diagram (Tsim has better functionality for that)
stim_circ.diagram(height=400)

# sampling using Stim
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=100)

### Using and adjusting heuristic noise model

In [ ]:
# Scaling mask for correlated CZ error rates of heuristic noise model
def make_scaled_cz_correlated_rates(s: float = 1.0) -> np.ndarray:
    """
    Scale the default correlated CZ error rates by factor `s`.
    
    The II (identity-identity) entry represents "no error", all other entries
    are error probabilities. We scale the errors and adjust II to maintain normalization.
    """
    # Default rates from the noise model
    default_rates = np.array([
        [9.93492628e-01, 2.27472300e-04, 2.27472300e-04, 1.51277730e-03],
        [2.27472300e-04, 1.42864200e-04, 1.42864200e-04, 1.43082900e-04],
        [2.27472300e-04, 1.42864200e-04, 1.42864200e-04, 1.43082900e-04],
        [1.51277730e-03, 1.43082900e-04, 1.43082900e-04, 1.42813990e-03],
    ])
    
    # Scale all error entries
    scaled_rates = default_rates.copy()
    scaled_rates[0, 0] = 0  # temporarily zero out II
    scaled_rates = scaled_rates * s  # scale all errors
    
    # Recalculate II to maintain normalization (sum = 1)
    scaled_total_error = scaled_rates.sum()
    scaled_rates[0, 0] = 1.0 - scaled_total_error
    
    # Clamp to valid probability range
    if scaled_rates[0, 0] < 0:
        raise ValueError(f"Scaling factor {s} too large - total error probability exceeds 1.0")
    
    return scaled_rates
def make_error_model(s: float = 1.0) -> dict[str, float]:
    """
    Create a dictionary of error-model parameters scaled by factor `s`.

    Parameters
    ----------
    s : float
        Global scaling factor applied multiplicatively to all nonzero
        error probabilities.

    Returns
    -------
    dict[str, float]
        Dictionary of scaled error-model parameters.
    """
    return {
        # Local errors
        "local_px": 0.0004102 * s,
        "local_py": 0.0004102 * s,
        "local_pz": 0.0004112 * s,
        "local_loss_prob": 0.0,

        "local_unaddressed_px": 2e-7 * s,
        "local_unaddressed_py": 2e-7 * s,
        "local_unaddressed_pz": 1.2e-6 * s,
        "local_unaddressed_loss_prob": 0.0,

        # Global errors
        "global_px": 0.000065 * s,
        "global_py": 0.000065 * s,
        "global_pz": 0.000065 * s,
        "global_loss_prob": 0.0,

        # CZ gates
        "cz_paired_gate_px": 0.0006549 * s,
        "cz_paired_gate_py": 0.0006549 * s,
        "cz_paired_gate_pz": 0.003184 * s,
        "cz_gate_loss_prob": 0.0,

        "cz_unpaired_gate_px": 0.0005149 * s,
        "cz_unpaired_gate_py": 0.0005149 * s,
        "cz_unpaired_gate_pz": 0.002185 * s,
        "cz_unpaired_loss_prob": 0.0,

        # Motion-related
        "mover_px": 0.000806 * s,
        "mover_py": 0.000806 * s,
        "mover_pz": 0.002458 * s,
        "move_loss_prob": 0.0,

        "sitter_px": 0.0003066 * s,
        "sitter_py": 0.0003066 * s,
        "sitter_pz": 0.0004639 * s,
        "sit_loss_prob": 0.0,
    }

In [ ]:
from bloqade.cirq_utils import noise
from bloqade.cirq_utils.emit import emit_circuit
from bloqade.cirq_utils import load_circuit

# applying noise model to circuit according to different architectures; must use Cirq circuit
@squin.kernel
def main():
    my_kernel()

cirq_main = emit_circuit(main) # emit to Cirq
noise_model = noise.GeminiOneZoneNoiseModel() # define noise model; here OneZone default
main_noisy = noise.transform_circuit(cirq_main, model=noise_model) # annotate circuit w noise
squin_main = load_circuit(main_noisy) #back to Squin
noisy_stim = to_stim(squin_main) #now to Stim


# Example: defining new noise model on top of OneZone architecture
noise_scale = 1
noise_model = noise.GeminiOneZoneNoiseModel(cz_paired_correlated_rates= make_scaled_cz_correlated_rates(noise_scale), **make_error_model(noise_scale))


Notice that Tsim may perform better than Stim as a backend sampler when using the automatized heuristic noise models, given rounding errors.